In [4]:
import os
import pandas as pd

data_path = os.path.join("..", "data", "raw", "380K_US_Restaurants.csv")

In [9]:
sample = pd.read_csv(data_path, nrows=5)
print(sample.columns)
sample

Index(['Title', 'Link', 'Category', 'Rating', 'Website', 'Phone', 'Address',
       'Images', 'Categories', 'Geo_Coordinates', 'Time_Zone', 'Latitude',
       'Longitude'],
      dtype='object')


,Title,Link,Category,Rating,Website,Phone,Address,Images,Categories,Geo_Coordinates,Time_Zone,Latitude,Longitude
0,Dairy Queen Grill & Chill,https://www.google.com/maps/place/Dairy+Queen+...,Fast food restaurant,3.8,http://www.fourteenfoods.com/?y_source=1_ODk5N...,+1256-496-0404,"Dairy Queen Grill & Chill, 3143 US-280, Alexan...",[{'thumbnail': 'https://lh5.googleusercontent....,"['Fast food restaurant', 'Ice cream shop']","{'latitude': 32.9338695, 'longitude': -85.9704...",America/Chicago,32.933870,-85.970419
1,Jake's Restaurant,https://www.google.com/maps/place/Jake%27s+Res...,American restaurant,4.4,http://jakesonbroad.com/,+1256-234-4300,"Jake's Restaurant, 16 Broad St, Alexander City...",[{'thumbnail': 'https://lh5.googleusercontent....,['American restaurant'],"{'latitude': 32.945406, 'longitude': -85.953806}",America/Chicago,32.945406,-85.953806
2,Carib Kitchen,https://www.google.com/maps/place/Carib+Kitche...,Caribbean restaurant,4.9,https://carib-kitchen.webnode.com/,+1256-392-4433,"Carib Kitchen, 68 Broad St, Alexander City, AL...",[{'thumbnail': 'https://lh5.googleusercontent....,['Caribbean restaurant'],"{'latitude': 32.9446173, 'longitude': -85.954932}",America/Chicago,32.944617,-85.954932
3,Cazadores Mexican Restaurant,https://www.google.com/maps/place/Cazadores+Me...,Mexican restaurant,4.5,NaN,+1256-392-3991,"Cazadores Mexican Restaurant, 910 Cherokee Rd,...",[{'thumbnail': 'https://lh5.googleusercontent....,['Mexican restaurant'],"{'latitude': 32.935643, 'longitude': -85.952365}",America/Chicago,32.935643,-85.952365
4,La Posada Mexican Grill,https://www.google.com/maps/place/La+Posada+Me...,Mexican restaurant,4.3,http://www.laposadamexicangrill.net/,+1256-329-3005,"La Posada Mexican Grill, 3714 US-280, Alexande...",[{'thumbnail': 'https://lh5.googleusercontent....,"['Mexican restaurant', 'Latin American restaur...","{'latitude': 32.926932, 'longitude': -85.9649187}",America/Chicago,32.926932,-85.964919
